In [ ]:
!pip install newspaper3k
!pip install xmltodict

# Fetch from Google news API

In [ ]:
import requests
import xmltodict

categories = ["politics", "health", "entertainment", "sports", "finance"]
articles = {}
for category in categories:
    news_url = f'https://news.google.com/rss/search?cf=all&hl=en-IN&pz=1&q={category}&csed=in&csep=false&num=100&sort=rated&gl=IN&ceid=IN:en'
    r = requests.get(news_url)
    data_dict = xmltodict.parse(r.content)
    articles[category] = data_dict['rss']['channel']['item']

In [ ]:
articles['health'][0]

OrderedDict([('title',
              'Here’s how ginger water can do wonders to your health - Hindustan Times'),
             ('link',
              'https://www.hindustantimes.com/lifestyle/health/heres-how-ginger-water-can-do-wonders-to-your-health-101630926650000.html'),
             ('guid',
              OrderedDict([('@isPermaLink', 'false'),
                           ('#text',
                            'CAIiEPSwIq5b4vL17OQ88shiCcQqFwgEKg4IACoGCAoww7k_MMevCDC88PkG')])),
             ('pubDate', 'Mon, 06 Sep 2021 11:15:49 GMT'),
             ('description',
              '<a href="https://www.hindustantimes.com/lifestyle/health/heres-how-ginger-water-can-do-wonders-to-your-health-101630926650000.html" target="_blank">Here’s how ginger water can do wonders to your health</a>&nbsp;&nbsp;<font color="#6f6f6f">Hindustan Times</font>'),
             ('source',
              OrderedDict([('@url', 'https://www.hindustantimes.com'),
                           ('#text', 'Hindustan Times

In [ ]:
data = []
for k in articles:
    for a in articles[k]:
        obj = {}
        obj['headline'] = a['title']
        obj['url'] = a['link']
        obj['source'] = a['source']['#text']
        obj['source_url'] = a['source']['@url']
        obj['category'] = k
        data.append(obj)

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
print(df.shape)
df.head()

(500, 5)


,headline,url,source,source_url,category
0,Rahul Gandhi 'political cuckoo' of Indian poli...,https://timesofindia.indiatimes.com/india/rahu...,Times of India,https://timesofindia.indiatimes.com,politics
1,Canada polls: ‘Parties must reject vote-based ...,https://www.hindustantimes.com/world-news/cana...,Hindustan Times,https://www.hindustantimes.com,politics
2,‘Take back power’: Talawa’s Run It Back and th...,https://www.theguardian.com/stage/2021/sep/06/...,The Guardian,https://www.theguardian.com,politics
3,Politics Hots Up Over Odisha’s Second AIIMS In...,https://odishatv.in/news/politics/politics-hot...,OTV News,https://odishatv.in,politics
4,"Summoned by ED, Abhishek leaves for Delhi, say...",https://indianexpress.com/article/cities/kolka...,The Indian Express,https://indianexpress.com,politics


In [ ]:
df.category.value_counts()

entertainment    100
sports           100
finance          100
politics         100
health           100
Name: category, dtype: int64

In [ ]:
df.to_csv('news_articles.csv')

# Scrape content using newspaper3k lib

In [ ]:
from newspaper import Article
text = []
cnt = 0
tot = 0
for i,url in enumerate(df['url']):
    if i%100 == 0:
        cnt = 0
        tot = 0
    if tot >= 20:
        text.append('')
        continue
    txt = ''
    try:
        article = Article(url)
        article.download()
        article.parse()
        txt = article.text
        tot += 1
    except:
        cnt+=1
        print(cnt)
    text.append(txt)
df['text'] = text

1
2
3
4
1
1
2
3
4
1
2
3
4
5
6


In [ ]:
len(text)

500

In [ ]:
df.head()

,headline,url,source,source_url,category,text
0,Manappuram Finance to consider fund raising op...,https://www.business-standard.com/article/news...,Business Standard,https://www.business-standard.com,finance,Manappuram Finance said the company may consid...
1,PayU aims to provide full-stack financial serv...,https://www.business-standard.com/article/comp...,Business Standard,https://www.business-standard.com,finance,"Finance, a digital lending player backed by Pr..."
2,L&T Finance in talks with HSBC to sell its mut...,https://www.businesstoday.in/latest/corporate/...,Business Today,https://www.businesstoday.in,finance,L&T Finance Holdings is in active talks with f...
3,Tally Solutions seeks small finance banking li...,https://economictimes.indiatimes.com/industry/...,Economic Times,https://economictimes.indiatimes.com,finance,Financial technology company Tally Solutions a...
4,China Finance Online announces changes to the ...,https://ibsintelligence.com/ibsi-news/china-fi...,IBS Intelligence,https://ibsintelligence.com,finance,"China Finance Online Co. Limited, a web-based ..."


In [ ]:
df_txt = df[df.text != '']
print(df_txt.shape)
df_txt.head()

(100, 6)


,headline,url,source,source_url,category,text
0,Manappuram Finance to consider fund raising op...,https://www.business-standard.com/article/news...,Business Standard,https://www.business-standard.com,finance,Manappuram Finance said the company may consid...
1,PayU aims to provide full-stack financial serv...,https://www.business-standard.com/article/comp...,Business Standard,https://www.business-standard.com,finance,"Finance, a digital lending player backed by Pr..."
2,L&T Finance in talks with HSBC to sell its mut...,https://www.businesstoday.in/latest/corporate/...,Business Today,https://www.businesstoday.in,finance,L&T Finance Holdings is in active talks with f...
3,Tally Solutions seeks small finance banking li...,https://economictimes.indiatimes.com/industry/...,Economic Times,https://economictimes.indiatimes.com,finance,Financial technology company Tally Solutions a...
4,China Finance Online announces changes to the ...,https://ibsintelligence.com/ibsi-news/china-fi...,IBS Intelligence,https://ibsintelligence.com,finance,"China Finance Online Co. Limited, a web-based ..."


In [ ]:
df_txt.to_csv('news_articles_txt100.csv')

In [ ]:
df_txt.category.value_counts()

politics         20
health           20
entertainment    20
sports           20
finance          20
Name: category, dtype: int64